In [2]:
# create table tt_q_tag_cnt as 
# select x.id
# 	, to_char(x.creationdate, 'yyyy-mm-dd') as c_date
# 	, (regexp_matches(x.tags, '<(ruby|scala|rust|go|bash|shell|objective-c|python|r|typescript|swift|c#|sql|assembly|kotlin|c\+\+|javascript|vba|c|html|css|java|perl|haskell|julia|php|dart|clojure|f#|elixir|erlang|lisp|groovy|crystal-lang|powershell|cobol|delphi|matlab)>', 'g'))[1] AS matched_tags
# 	, x.tag
#   from (
# 	select id, creationdate, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag 
#   from posts
# where posttypeid ='1'
#   and (tags like '%<ruby>%' or tags like '%<scala>%' or tags like '%<rust>%' or tags like '%<go>%' or tags like '%<bash>%' or tags like '%<shell>%' or tags like '%<objective-c>%' or tags like '%<python>%' or tags like '%<r>%' or tags like '%<typescript>%' or tags like '%<swift>%' or tags like '%<c#>%' or tags like '%<sql>%' or tags like '%<assembly>%' or tags like '%<kotlin>%' or tags like '%<c++>%' or tags like '%<javascript>%' or tags like '%<vba>%' or tags like '%<c>%' or tags like '%<html>%' or tags like '%<css>%' or tags like '%<java>%' or tags like '%<perl>%' or tags like '%<haskell>%' or tags like '%<julia>%' or tags like '%<php>%' or tags like '%<dart>%' or tags like '%<clojure>%' or tags like '%<f#>%' or tags like '%<elixir>%' or tags like '%<erlang>%' or tags like '%<lisp>%' or tags like '%<groovy>%' or tags like '%<crystal-lang>%' or tags like '%<powershell>%' or tags like '%<cobol>%' or tags like '%<delphi>%' or tags like '%<matlab>%')
#   and creationdate between '2021-11-30' and '2023-12-03' 
#   ) x 
# ;

In [3]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
ppp = os.path.abspath('../../visualization')

sys.path.insert(1, p)
sys.path.insert(1, pp)
sys.path.insert(1, ppp)



In [4]:
import json
import pandas as pd
import numpy as np
from utils.salary import *
import psycopg2
from utils.statistics import *
import config.config as conf
import datetime
import re
from statsmodels.nonparametric.smoothers_lowess import lowess
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
colors = plt.get_cmap('tab20').colors 

ModuleNotFoundError: No module named 'utils'

In [ ]:
from matplotlib import font_manager
import matplotlib as mpl
from matplotlib import font_manager as fm

# 설치된 폰트 이름과 경로 확인
for font in font_manager.fontManager.ttflist:
    if 'Helvetica' in font.name: # 'Nanum' 대신 원하는 글꼴의 일부를 입력하여 검색할 수 있습니다.
        print(font.name, font.fname)

# 직접 경로로 Helvetica 폰트 불러오기
font_path = "/System/Library/Fonts/Helvetica.ttc"
font_prop = fm.FontProperties(fname=font_path)
font_name = font_prop.get_name()
print(f"Registered font name: {font_name}")

mpl.rcParams['font.family'] = font_name


## 데이터전처리

In [ ]:
output_dir = './fig/'
by = 'rel_week'

In [ ]:
# test.json 내용: [{"name":"Jack","age":26},{"name":"Ace","age":87}]
df = pd.read_json('../../visualization/result/tag/salary.json') ## pd.read_json 이용

In [ ]:
df = df.reset_index().rename(columns={'index':'year'})

In [ ]:
df = df.sort_values(by = 'year').reset_index(drop=True)

In [ ]:
df

In [ ]:
def get_CID_ratio_df(df, year) : 
    year_list = [year-3, year-2, year-1, year]
    print(year_list)
    df = df = df[df['year'].isin(year_list)]
    df_salary = df.melt(id_vars = 'year', value_vars = df.columns[1:], var_name = 'lang', value_name = 'salary')
    df_salary = df_salary.dropna()

    filtered_df = df_salary[df_salary['lang'].isin(df_salary['lang'].value_counts()[lambda x: x == len(year_list)].index)].copy()
    filtered_df['salary_ratio'] = filtered_df.groupby(['lang'])['salary'].transform(lambda x: x/x.shift(1) )
    filtered_df = filtered_df.dropna()

    filtered_df.loc[ : , 'CID'] = filtered_df.sort_values(by = ['lang', 'year'], ascending=[True, True]).groupby(['lang'])['salary_ratio'].diff()
    filtered_df = filtered_df.dropna().reset_index(drop=True)

    filtered_df.loc[ : , 'CID'] = filtered_df.sort_values(by = ['lang', 'year'], ascending=[True, True]).groupby(['lang'])['CID'].diff()
    filtered_df = filtered_df.dropna().reset_index(drop=True)


    return filtered_df




In [ ]:
def get_CID_df(df, year) : 
    year_list = [year-2, year-1, year]
    print(year_list)
    df = df = df[df['year'].isin(year_list)]
    df_salary = df.melt(id_vars = 'year', value_vars = df.columns[1:], var_name = 'lang', value_name = 'salary')
    df_salary = df_salary.dropna()

    filtered_df = df_salary[df_salary['lang'].isin(df_salary['lang'].value_counts()[lambda x: x == len(year_list)].index)].copy()
    filtered_df.loc[ : , 'CID'] = filtered_df.sort_values(by = ['lang', 'year'], ascending=[True, True]).groupby(['lang'])['salary'].diff()
    filtered_df = filtered_df.dropna().reset_index(drop=True)

    filtered_df.loc[ : , 'CID'] = filtered_df.sort_values(by = ['lang', 'year'], ascending=[True, True]).groupby(['lang'])['CID'].diff()
    filtered_df = filtered_df.dropna().reset_index(drop=True)


    return filtered_df




In [ ]:
def get_CID_log_df(df, year) : 
    year_list = [year-2, year-1, year]
    print(year_list)
    df = df = df[df['year'].isin(year_list)]
    df_salary = df.melt(id_vars = 'year', value_vars = df.columns[1:], var_name = 'lang', value_name = 'salary')
    df_salary = df_salary.dropna()
    df_salary['salary'] = np.log(df_salary['salary'])

    filtered_df = df_salary[df_salary['lang'].isin(df_salary['lang'].value_counts()[lambda x: x == len(year_list)].index)].copy()
    filtered_df.loc[ : , 'CID'] = filtered_df.sort_values(by = ['lang', 'year'], ascending=[True, True]).groupby(['lang'])['salary'].diff()
    filtered_df = filtered_df.dropna().reset_index(drop=True)

    filtered_df.loc[ : , 'CID'] = filtered_df.sort_values(by = ['lang', 'year'], ascending=[True, True]).groupby(['lang'])['CID'].diff()
    filtered_df = filtered_df.dropna().reset_index(drop=True)


    return filtered_df




In [ ]:
def get_CID_ratio_rel_df(df, year) : 
    year_list = [year-3, year-2, year-1, year]
    print(year_list)
    df = df = df[df['year'].isin(year_list)]
    df_salary = df.melt(id_vars = 'year', value_vars = df.columns[1:], var_name = 'lang', value_name = 'salary')
    df_salary = df_salary.dropna()

    valid_langs = df_salary['lang'].value_counts()[lambda x: x == len(year_list)].index
    filtered_df = df_salary[df_salary['lang'].isin(valid_langs)].copy()

    # 언어별로 연도 정렬 후 상대 변화율(성장률) 계산
    filtered_df.sort_values(by=['lang', 'year'], inplace=True)
    filtered_df['growth_rate'] = filtered_df.groupby('lang')['salary'].pct_change()

    # growth_rate의 이차 차분 = CID 계산
    filtered_df['CID'] = filtered_df.groupby('lang')['growth_rate'].diff()
    filtered_df['CID'] = filtered_df.groupby('lang')['CID'].diff()

    # 최종 정리: CID 계산이 가능한 연도만 남김
    filtered_df = filtered_df.dropna(subset=['growth_rate', 'CID']).reset_index(drop=True)


    return filtered_df




In [ ]:
def filter_df( df, type ) :
    tot_filtered_df = pd.DataFrame()
    for year in [ 2022, 2023, 2024] : 
        if type == 'ratio' : 
            filtered_df = get_CID_ratio_df(df, year)
        elif type == 'log' : 
            filtered_df = get_CID_log_df(df, year)
        elif type == 'ratio_rel' :
            filtered_df = get_CID_ratio_rel_df(df, year)
            
        else :
            filtered_df = get_CID_df(df, year)
            
        tot_filtered_df = pd.concat([tot_filtered_df, filtered_df], axis = 0)
    
        tot_exist_df = tot_filtered_df[tot_filtered_df['lang'].isin(tot_filtered_df['lang'].value_counts()[lambda x: x == len([ 2022, 2023, 2024])].index)].copy()
        tot_exist_df=  tot_exist_df.sort_values(by = ['year', 'CID'], ascending=[False, True])
    return tot_exist_df

In [ ]:
tot_df_r = filter_df(df, 'ratio')
tot_df_log = filter_df(df, 'log')
tot_df_nr = filter_df(df, 'notratio')
tot_df_rl = filter_df(df, 'ratio_rel')

In [ ]:
cid_2024_lang = tot_df_rl[tot_df_rl['year'] == 2024].sort_values(by = ['CID'], ascending=[False])

In [ ]:
cid_2024_df = tot_df_rl[tot_df_rl['lang'].isin(cid_2024_lang['lang'].values)]

In [ ]:
cid_2024_df.head()
cid_2024 = cid_2024_df[cid_2024_df['year'] == 2024].sort_values(by = ['CID'], ascending=[False])

In [ ]:
cid_2024

## 데이터 로드

In [ ]:
# conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
# try:
#     cur = conn.cursor()
#     cur.execute("\
# select c_date, matched_tags, tag, count(*) as cnt \
#   from tt_q_tag_cnt \
#  group by c_date, matched_tags, tag \
# ; \
#                 " 
#    )
#     rows = cur.fetchall()
    

# except psycopg2.DatabaseError as db_err:
#     print(db_err)
# finally : 
#   cur.close()

In [ ]:
# conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
# try:
#     cur = conn.cursor()
#     cur.execute("\
# select c_date, matched_tags, tag, sum(prop_for_t) as cnt \
#   from tt_q_tag_cnt_02 \
#  group by c_date, matched_tags, tag \
# ; \
#                 " 
#    )
#     rows = cur.fetchall()
    

# except psycopg2.DatabaseError as db_err:
#     print(db_err)
# finally : 
#   cur.close()

In [ ]:
str([x for x in cid_2024['lang']])[1:-1]

In [ ]:
q_string =  f"select c_date, matched_tags, tag, sum(prop_for_t) as cnt from tt_q_tag_cnt_02 where matched_tags in ({str([x for x in cid_2024['lang']])[1:-1]}) group by c_date, matched_tags, tag; "


In [ ]:
q_string

In [ ]:
# CID를 다시 계산함에 따라 시각화에 사용할 언어 변경(2020년부터 2024년까지 모든 SALARY 데이터가 존재하는 언어를 대상으로 )
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_string)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
tag_dis_df = pd.DataFrame(rows, columns = [
  'cdate' 
, 'matched_tags' 
, 'tag'
, 'cnt'
])

In [ ]:
tag_dis_df.head()

In [ ]:
std_date = datetime.datetime(2022, 11, 30)

In [ ]:
tag_dis_df['cdate'] = pd.to_datetime(tag_dis_df['cdate'], format="%Y-%m-%d")
tag_dis_df['rel_day'] = (tag_dis_df['cdate'] - std_date).dt.days
tag_dis_df['rel_week'] = np.floor(tag_dis_df['rel_day']/7)

In [ ]:
tag_dis_df = tag_dis_df[(tag_dis_df['rel_week']>-53 ) & (tag_dis_df['rel_week']<=52 )]

In [ ]:
tot_df = tag_dis_df.groupby([by, 'matched_tags'])['cnt'].sum().reset_index(name='tot_cnt')

In [ ]:
tag_dis_by_df = tag_dis_df.groupby([by, 'matched_tags', 'tag'])['cnt'].sum().reset_index(name = 'cnt')

In [ ]:
tag_dis_by_df.head()

In [ ]:
df_proportion = pd.merge(tag_dis_by_df, tot_df, on = [by, 'matched_tags'], how = 'left')
df_proportion['proportion'] = df_proportion['cnt'] / df_proportion['tot_cnt']
# .reset_index(name='count')

In [ ]:
df_proportion.tail()

In [ ]:
df_proportion['proportion'] = df_proportion['proportion'].astype(float)
df_proportion['tot_cnt'] = df_proportion['tot_cnt'].astype(float)

In [ ]:
df_entropy = df_proportion.groupby([by, 'matched_tags'])['proportion'].apply(lambda x : calculate_entropy(x)).to_frame(name='entropy').reset_index()

In [ ]:
df_entropy_viz = pd.merge(df_entropy, df_proportion[[by, 'matched_tags', 'tot_cnt']].drop_duplicates(), \
                           on = [by, 'matched_tags'], how = 'left')

In [ ]:
df_proportion

In [ ]:
df_entropy_viz['log_tot_cnt'] = np.log(df_entropy_viz['tot_cnt'])

In [ ]:
cid_2024.shape

In [ ]:
lang_t5_2024 = list(cid_2024['lang'])[:5]
lang_b5_2024 = list(cid_2024['lang'])[-5:]

In [ ]:
# lang_t5_2024.append('python')
# # lang_b5_2024.append('python')

In [ ]:
df_entropy_viz_bf = df_entropy_viz[df_entropy_viz[by]<0]
df_entropy_viz_af = df_entropy_viz[df_entropy_viz[by]>=0]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu

def analyze_entropy_change(data, measure='entropy'):
    results = []

    languages = data['matched_tags'].unique()

    for lang in languages:
        df_lang = data[data['matched_tags'] == lang]
        before = df_lang[df_lang['rel_week'] < 0]['entropy']
        after  = df_lang[df_lang['rel_week'] >= 0]['entropy']
        

        if len(before) < 5 or len(after) < 5:
            continue  # 샘플 너무 적으면 스킵

        # 평균
        mean_before = before.mean()
        mean_after = after.mean()
        diff = mean_after - mean_before

        # 표준오차 (SE)와 95% 신뢰구간
        pooled_se = np.sqrt(before.var(ddof=1)/len(before) + after.var(ddof=1)/len(after))
        ci_low = diff - 1.96 * pooled_se
        ci_high = diff + 1.96 * pooled_se

        pval = mannwhitneyu(before, after, alternative='two-sided').pvalue

        results.append({
            'Language': lang,
            'Mean_Before': mean_before,
            'Mean_After': mean_after,
            'Diff': diff,
            'CI_Low': ci_low,
            'CI_High': ci_high,
            'pvalue': pval
        })

    summary_df = pd.DataFrame(results)
    return summary_df

In [ ]:
marker_label = {
    "High": "^",    # 원
    "Mid": "s",     # 사각형
    "Low": "o"      # 삼각형
}

color_label = {
    "High": "#e31a1c",
    "Mid": "#333333",
    "Low": "#1f78b4"
}


In [ ]:
import matplotlib.pyplot as plt

def plot_forest_with_significance(summary_df):
    summary_df = summary_df.sort_values('Diff')
    languages = summary_df['Language']
    diffs = summary_df['Diff'].values
    ci_lows = diffs - summary_df['CI_Low'].values
    ci_highs = summary_df['CI_High'].values - diffs
    p_values = summary_df['pvalue'].values

    # fig, ax = plt.subplots(figsize=(12, max(6, len(languages) * 0.45)), constrained_layout=True)
    fig, ax = plt.subplots(figsize=(9, max(6, len(languages) * 0.45)), constrained_layout=True)

        # 별표 추가
    significance = []
    for p in p_values:
        if p < 0.05:
            significance.append('*')
        else:
            significance.append('')

    labels = [f"{lang} ({sig})" if sig else lang for lang, sig in zip(languages, significance)]

    color_map = {}
    marker_map = {}
    for label in labels:
        original_text = label
        clean_text = original_text.replace('*', '').replace('(', '').replace(')', '').replace(' ', '')  # "python" 으로 비교
        if clean_text in lang_t5_2024:
            color_map[label]  = color_label['High']  # 강조할 언어는 빨간색
            marker_map[label] = marker_label['High']  # 강조할 언어는 빨간색
        elif clean_text in lang_b5_2024:
            color_map[label]  = color_label['Low'] 
            marker_map[label] = marker_label['Low']
        else:
            color_map[label]  = color_label['Mid'] 
            marker_map[label] = marker_label['Mid']

    # 데이터 루프
    for diff, label, ci_low, ci_high in zip(diffs, labels, ci_lows, ci_highs):
        c = color_map.get(label, "#333333")  # 매핑 없으면 기본은 검정
        err = np.array([[ci_low], [ci_high]])
        ax.errorbar(
            diff, label,
            xerr=err,
            # fmt='o',
            color=c,
            ecolor=c,
            elinewidth=2,
            capsize=4,
            fmt=marker_map.get(label, 'o'),  # 매핑 없으면 기본은 원형
            markersize=10
        )
        

    
    # ax.errorbar(diffs, labels, xerr=errors, fmt='o', color='#333333', ecolor='#333333', elinewidth=2, capsize=4)
    ax.axvline(0, color='black', linestyle='-')
    # ax.set_title('Language-wise Entropy Change')
    ax.set_xlabel('Change in Entropy(95% CI)', fontsize=20)
    ax.grid(True, axis='x')

    plt.tick_params(axis='x', labelsize=16)
    plt.tick_params(axis='y', labelsize=16)

    for label in ax.get_yticklabels():
        original_text = label.get_text()        # 예: "python***"
        label.set_color(color_map[original_text])  # 강조할 언어는 빨간색

    handles = [
    Line2D([0], [0], marker=marker_label["Low"],  color=color_label["Low"],  linestyle="",
        markersize=7, label="Low CID"),
    Line2D([0], [0], marker=marker_label["Mid"],  color=color_label["Mid"],  linestyle="",
        markersize=7, label="Mid CID"),
    Line2D([0], [0], marker=marker_label["High"], color=color_label["High"], linestyle="",
        markersize=7, label="High CID")
    ]
    fig.legend(
        handles=handles,
        # labels=labels,
        loc = 'lower center',
        bbox_to_anchor=(0.5, -0.1),   # 캔버스 밖으로 살짝 내리기
        frameon=False, 
        ncol=3,
        fontsize=20,
        # handler_map={Line2D: HandlerLine2D(numpoints=1)},  # 마커 1개만 일관되게
        # handlelength=1.0, handleheight=1.0,  # 마커 상자 크기 통일
        # handletextpad=0.6, columnspacing=2.0, labelspacing=0.8,
        # borderaxespad=0.0
    )
    plt.savefig(f"{output_dir}Result_Fig13.png", dpi=300, bbox_inches='tight')
    plt.show()


In [ ]:
# (1) 데이터 준비
# data에는 'matched_tags' (언어명), 'period' (before/after), 'entropy' 컬럼이 있어야 해

# (2) 분석
summary_df = analyze_entropy_change(df_entropy_viz, measure='entropy')

plot_forest_with_significance(summary_df)

# (4) Summary 테이블도 확인 가능
display(summary_df)


In [ ]:
summary_df = summary_df.sort_values('Diff')
top_summary_df = summary_df[summary_df['Language'].isin(lang_t5_2024)]
bot_summary_df = summary_df[summary_df['Language'].isin(lang_b5_2024)]
mid_summary_df = summary_df[~(summary_df['Language'].isin(lang_t5_2024)) & ~(summary_df['Language'].isin(lang_b5_2024))]

In [ ]:
summary_dict = dict()
summary_dict['High'] = top_summary_df
summary_dict['Mid'] = mid_summary_df
summary_dict['Low'] = bot_summary_df

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True, sharex=True)

for i, key in enumerate(summary_dict.keys()):
    s_df        = summary_dict[key]
    languages   = s_df['Language']
    diffs       = s_df['Diff'].values
    ci_lows     = diffs - s_df['CI_Low'].values
    ci_highs    = s_df['CI_High'].values - diffs
    p_values    = s_df['pvalue'].values


    significance = []
    for p in p_values:
        if p < 0.05:
            significance.append('*')
        else:
            significance.append('')

    labels = [f"{lang} ({sig})" if sig else lang for lang, sig in zip(languages, significance)]

    for diff, ci_low, ci_high, label, si in zip(diffs , ci_lows, ci_highs, labels, significance):
        print(si)
        err = np.array([[ci_low], [ci_high]])
        axs[i].errorbar(
            diff, label,
            xerr=err,
            fmt=marker_label[key],
            color= color_label[key],
            ecolor=color_label[key],
            elinewidth=2,
            capsize=4,
            markersize=6
        )
    # ax.errorbar(diffs, labels, xerr=errors, fmt='o', color='#333333', ecolor='#333333', elinewidth=2, capsize=4)
    axs[i].set_title(f"{key} CID", fontsize=25)
    axs[i].axvline(0, color='black', linestyle='-')
    axs[i].tick_params(axis='x', labelsize=16)
    axs[i].tick_params(axis='y', labelsize=16)
    axs[i].grid(True, axis='x')
    fig.supxlabel('Change in Entropy(95% CI)', fontsize=20)
plt.savefig(f"{output_dir}Result_Fig13_1.png", dpi=300, bbox_inches='tight')
plt.show()
    


In [ ]:
lang_t5_2024
lang_b5_2024

In [ ]:
res_summary_df = summary_df[~(summary_df['Language'].isin(lang_t5_2024)) & ~(summary_df['Language'].isin(lang_b5_2024))]

In [ ]:
top_summary_df

In [ ]:
bot_summary_df